In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import kale
import kale.serialize

# Construct Workflow

In [ ]:
def greet(salutation, name):
    print("{}, {}!".format(salutation, name))

In [ ]:
wp = kale.workflow_objects.WorkerPool(
    'WP', 
    wf_executor='parsl',
    num_workers=4
)
wf = kale.workflow_objects.Workflow("WF")
t1 = kale.workflow_objects.PythonFunctionTask(
    name='hello',
    func=greet,
    args=["Hello, there", "Bob"]
)
t2 = kale.workflow_objects.PythonFunctionTask(
    name='howdy',
    func=greet,
    args=["Howdy", "partner"]
)
t3 = kale.workflow_objects.PythonFunctionTask(
    name='heyo',
    func=greet,
    args=["Heyo", "Guy"]
)
t4 = kale.workflow_objects.PythonFunctionTask(
    name='bye',
    func=greet,
    args=["So long", "friend"]
)
wf.add_task(t1)
wf.add_task(t2, dependencies=[t1])
wf.add_task(t3, dependencies=[t1])
wf.add_task(t4, dependencies=[t2,t3])

In [ ]:
wpw = kale.workflow_widgets.WorkerPoolWidget()

In [ ]:
kale.workflow_widgets.WorkflowWidget(wf, wpw)

# Serialize and Submit

In [ ]:
import flask
import dill

In [ ]:
wf_bytes = kale.serialize.serialize_wf(wf)

In [ ]:
wf_dict = dill.loads(wf_bytes)
#wf_dict

In [ ]:
r = requests.post(
    "http://localhost:12643/submit_parsl",
    data=wf_bytes,
)

r

# Deserialize and Reconstruct
To demonstrate that it looks the same on the other side of the wire

In [ ]:
re_wf = kale.serialize.deserialize_wf(wf_bytes)
kale.workflow_widgets.WorkflowWidget(re_wf, wpw)